### 第四部分 面向对象惯用法
#### 第八章 对象引用、可变性和垃圾回收

##### 8.1 变量不是盒子
变量只不过是标注，所以无法阻止为对象贴上多个标注。贴的多个
标注，就是别名。

In [2]:
# 变量a和b引用同一个列表，而不是哪个列表的副本
a = [1, 2, 3]
b = a
a.append(4)
print(b)

[1, 2, 3, 4]


In [3]:
# 创建对象之后才会把变量分配给对象
class Gizmo:
    def __init__(self):
        print('Gizmo id: %d' % id(self))

In [4]:
x = Gizmo()

Gizmo id: 2219258039152


##### 8.2 标识、相等性和别名
== 运算符比较两个对象的值（对象中保存的数据），而 is 比较对象的
标识。

In [11]:
# charles 和 lewis 指代同一个对象
charles = {'name': 'Charles L. Dodgson', 'born': 1832}
lewis = charles
print(lewis is charles)
print(id(lewis), id(charles))
lewis['balance'] = 950
print(charles)

True
2219257810808 2219257810808
{'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}


In [15]:
alex = {'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}
print(alex == charles)
print(alex is charles)
print(id(alex), id(charles))

True
False
2219258392792 2219257810808


In [24]:
# 元组的值会随着应用的可变对象的变化而变，元组中不可变的是元素的标识
t1 = (1, 2, [30, 40])
t2 = (1, 2, [30, 40])
print(t1 == t2)
print(id(t1[-1]))
t1[-1].append(99)
print(t1)
print(id(t1[-1]))
print(t1 == t2)

True
2219258467336
(1, 2, [30, 40, 99])
2219258467336
False


##### 8.3 默认做浅复制
构造方法或 [:] 做的是浅复制（即复制了最外层容器，副本中
的元素是源容器中元素的引用）。如果所有元素都是不可变的，那么这
样没有问题，还能节省内存。但是，如果有可变的元素，可能就会导致
意想不到的问题。

In [26]:
# 为一个包含另一个列表的列表做浅复制
l1 = [3, [66, 55, 44], (7, 8, 9)]
l2 = list(l1)
l1.append(100)
l1[1].remove(55)
print('l1:', l1)
print('l2:', l2)
l2[1] += [33, 22]
l2[2] += (10, 11)
print('l1:', l1)
print('l2:', l2)

l1: [3, [66, 44], (7, 8, 9), 100]
l2: [3, [66, 44], (7, 8, 9)]
l1: [3, [66, 44, 33, 22], (7, 8, 9), 100]
l2: [3, [66, 44, 33, 22], (7, 8, 9, 10, 11)]


- copy 模块提供的 deepcopy 和 copy 函数能为任意对象做
深复制和浅复制。

In [2]:
# 校车乘客在途中上车和下车
class Bus:
    
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
    
    def pick(self, name):
        self.passengers.append(name)
    
    def drop(self, name):
        self.passengers.remove(name)

In [3]:
# 使用copy和deepcopy产生的影响
import copy
bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1)
bus3 = copy.deepcopy(bus1)
print(id(bus1), id(bus2), id(bus3))
bus1.drop('Bill')
print(bus2.passengers)
print(id(bus1.passengers), id(bus2.passengers), id(bus3.passengers))
print(bus3.passengers)

2648763767272 2648763767328 2648763767552
['Alice', 'Claire', 'David']
2648763760072 2648763760072 2648763757832
['Alice', 'Bill', 'Claire', 'David']


In [4]:
# 循环引用：b引用a,然后追加到a中：deepcopy会想办法复制a
a = [10, 20]
b = [a, 30]
a.append(b)
print(a)

[10, 20, [[...], 30]]


In [6]:
c = copy.deepcopy(a)
print(c)

[10, 20, [[...], 30]]


##### 8.4 函数的参数作为引用时
Python 唯一支持的参数传递模式是共享传参（call by sharing）。共享传参指函数的各个形式参数获得实参中各个引用的副本。也就是说，函数内部的形参是实参的别名。

In [7]:
# 函数可能会修改收到的任何可变对象
def f(a, b):
    a += b
    return a

In [11]:
x = 1
y = 2
print(f(x, y), x, y)
a = [1, 2]
b = [3, 4]
print(f(a, b), a, b)
t = (10, 20)
u = (30, 40)
print(f(t, u), t, u)

3 1 2
[1, 2, 3, 4] [1, 2, 3, 4] [3, 4]
(10, 20, 30, 40) (10, 20) (30, 40)


In [19]:
# 一个简单的类，说明可变默认值的危险
class HauntedBus:
    '''备受幽灵乘客折磨的校车'''
    def __init__(self, passengers=[]):
        self.passengers = passengers
        
    def pick(self, name):
        self.passengers.append(name)
    
    def drop(self, name):
        self.passengers.remove(name)

In [23]:
bus1 = HauntedBus(['Alice', 'Bill'])
print(bus1.passengers)
bus1.pick('Charles')
bus1.drop('Alice')
print(bus1.passengers)
bus2 = HauntedBus()
print(bus2.passengers)
bus2.pick('Carrie')
print(bus2.passengers)
bus3 = HauntedBus()
print(bus3.passengers)

['Alice', 'Bill']
['Bill', 'Charles']
['Carrie']
['Carrie', 'Carrie']
['Carrie', 'Carrie']


In [30]:
bus3.pick('Dave')
print(bus3.passengers)
print(bus2.passengers is bus3.passengers)
print(bus2.passengers)
print(dir(HauntedBus.__init__))
print(HauntedBus.__init__.__defaults__)

['Carrie', 'Carrie', 'Dave', 'Dave', 'Dave', 'Dave', 'Dave', 'Dave']
True
['Carrie', 'Carrie', 'Dave', 'Dave', 'Dave', 'Dave', 'Dave', 'Dave']
['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']
(['Carrie', 'Carrie', 'Dave', 'Dave', 'Dave', 'Dave', 'Dave', 'Dave'],)


In [31]:
print(HauntedBus.__init__.__defaults__[0] is bus2.passengers)

True


In [36]:
# 一个简单的类，说明接收可变参数的风险
class TwilightBus:
    '''让乘客销声匿迹的校车'''
    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)
    
    def pick(self, name):
        self.passengers.append(name)
    
    def drop(self, name):
        self.passengers.remove(name)

In [37]:
basketball_team = ['Sue', 'Tina', 'Maye', 'Diana', 'Pat']
bus = TwilightBus(basketball_team)
bus.drop('Tina')
bus.drop('Pat')
print(basketball_team)

['Sue', 'Tina', 'Maye', 'Diana', 'Pat']


##### 8.5 del和垃圾回收
del 语句删除名称，而不是对象。del 命令可能会导致对象被当作垃圾
回收，但是仅当删除的变量保存的是对象的最后一个引用，或者无法得
到对象时。 重新绑定也可能会导致对象的引用数量归零，导致对象被
销毁。

In [51]:
import weakref
s1 = {1, 2, 3}
s2 = s1
def bye():
    print('Gone  with the wind...')

ender = weakref.finalize(s1, bye)
print(ender.alive)
del s1
print(ender.alive)

Gone  with the wind...
True
True


In [52]:
s2 = 'spam' # 重新绑定
print(ender.alive)

Gone  with the wind...
False


##### 8.6 弱引用
弱引用不会增加对象的引用数量。引用的目标对象称为所指对象（referent）。因此我们说，弱引用不会妨碍所指对象被当作垃圾回收。

In [55]:
# 弱引用是可调用的对象，返回的是被引用的对象，如果所指对象不存在了，返回None
import weakref

a_set = {0, 1}
wref = weakref.ref(a_set)
print(wref)
print(wref())
a_set = {2, 3, 4}
print(wref())

<weakref at 0x00000268B9B611D8; to 'set' at 0x00000268B9AE2588>
{0, 1}
None


In [56]:
# Cheese 有个kind属性和标准的字符串表示形式
class Cheese:
    
    def __init__(self, kind):
        self.kind = kind
    
    def __repr__(self):
        return 'Cheese(%s)' % self.kind

In [60]:
import weakref

stock = weakref.WeakValueDictionary()
catalog = [Cheese('Red Leicester'), Cheese('Tilsit'),
           Cheese('Brie'), Cheese('Parmesan')]
for cheese in catalog:
    stock[cheese.kind] = cheese
print(sorted(stock.keys()))
del catalog
print(sorted(stock.keys()))
print(cheese.kind)
del cheese
print(sorted(stock.keys()))

['Brie', 'Parmesan', 'Red Leicester', 'Tilsit']
['Parmesan']
Parmesan
[]


#### 第九章 符合python风格的对象
##### 9.1 对象表示形式  
- repr()，以便于开发者理解的方式返回对象的字符串表示形式。
- str()，以便于用户理解的方式返回对象的字符串表示形式。
- \_\_repr\_\_、\_\_str\_\_ 和 \_\_format\_\_ 都必须返回 Unicode 字
符串（str 类型）。只有 \_\_bytes\_\_ 方法应该返回字节序列
（bytes 类型）。

##### 9.2 再谈向量类

In [1]:
from array import array
import math


class Vector2d:
    typecode = 'd'

    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    def __iter__(self):
        return (i for i in (self.x, self.y))

    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)

    def __str__(self):
        return str(tuple(self))

    def __bytes__(self):
        return (bytes([ord(self.typecode)]) +
                bytes(array(self.typecode, self)))
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    def __format__(self, fmt_spec=''):
        if fmt_spec.endswith('p'):
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
            outer_fmt = '<{}, {}>'
        else:
            coords = self
            outer_fmt = '({}, {})'
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)
    
    def __hash__(self):
        return hash(self.x) ^ hash(self.y)
    
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    def angle(self):
        return math.atan2(self.y, self.x)
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

##### 9.4 classmethod与staticmethod

In [32]:
# 比较classmethod和staticmethod的行为
class Demo:
    
    @classmethod
    def klassmeth(*args):
        return args
    
    @staticmethod
    def statmeth(*args):
        return args

In [33]:
print(Demo.klassmeth())
print(Demo.klassmeth('spam'))
print(Demo.statmeth())
print(Demo.statmeth('spam'))

(<class '__main__.Demo'>,)
(<class '__main__.Demo'>, 'spam')
()
('spam',)


##### 9.5 格式化显示
内置的 format() 函数和 str.format() 方法把各个类型的格式化方式
委托给相应的 .\_\_format\_\_(format_spec) 方法。format_spec 是格
式说明符，它是：
- format(my_obj, format_spec) 的第二个参数，或者
- str.format() 方法的格式字符串，{} 里代换字段中冒号后面的部
分

格式规范微语言为一些内置类型提供了专用的表示代码。比如，b 和 x
分别表示二进制和十六进制的 int 类型，f 表示小数形式的 float 类
型，而 % 表示百分数形式：

In [23]:
print(format(42, 'b'))
print(format(2/3, '.1%'))

101010
66.7%


In [24]:
from datetime import datetime

now = datetime.now()
print(format(now, '%H:%M:%S'))
print("It's now {:%I:%M %p}".format(now))

14:45:15
It's now 02:45 PM


In [2]:
v1 = Vector2d(3, 4)
print(format(v1))
print(format(v1, '.2f'))
print(format(v1, '.3e'))
print(format(v1, 'p'))
print(format(v1, '.3ep'))
print(format(v1, '.5p'))

(3.0, 4.0)
(3.00, 4.00)
(3.000e+00, 4.000e+00)
<5.0, 0.9272952180016122>
<5.000e+00, 9.273e-01>
<5.0, 0.9273>


##### 9.6 可散列的Vector2d

In [3]:
print(hash(v1))
print(set([v1]))

7
{Vector2d(3.0, 4.0)}


#####  9.7 Python 的私有属性和“受保护的”属性
如果以 \_\_mood 的形式（两个前导下划线，尾部没
有或最多有一个下划线）命名实例属性，Python 会把属性名存入实例的
\_\_dict\_\_ 属性中，而且会在前面加上一个下划线和类名。因此，对
Dog 类来说，\_\_mood 会变成 \_Dog\_\_mood；对 Beagle 类来说，会变成
\_Beagle\_\_mood。这个语言特性叫名称改写（name mangling）。

In [7]:
print(v1.__dict__)

{'_Vector2d__x': 3.0, '_Vector2d__y': 4.0}


##### 9.8 使用__slots__类属性节省空间
- 默认情况下，Python 在各个实例中名为\_\_dict\_\_ 的字典里存储实例属性。为了使用底层的散列表提升访问速度，字典会消耗大量内存。如果要处理数百万个属性不多的实例，通过 \_\_slots\_\_类属性，能节省大量内存，方法是让解释器在元组中存储实例属性，而不用字典。
- 定义 \_\_slots\_\_ 的方式是，创建一个类属性，使用 \_\_slots\_\_ 这个名字，并把它的值设为一个字符串构成的可迭代对象，其中各个元素表示各个实例属性。

##### 9.9 覆盖类属性

In [12]:
# 设定从类中继承的typecode属性，自定义一个实例属性
dumpd = bytes(v1)
print(len(dumpd), dumpd)
v1.typecode = 'f'
dumpd = bytes(v1)
print(len(dumpd), dumpd)
print(Vector2d.typecode)

9 b'f\x00\x00@@\x00\x00\x80@'
9 b'f\x00\x00@@\x00\x00\x80@'
d


In [17]:
# ShortVector2d是Vector2d的子类，只用于覆盖typecode的默认值
class ShorVector2d(Vector2d):
    typecode = 'f'

sv = ShorVector2d(1/11, 1/27)
print(repr(sv), len(bytes(sv)))

ShorVector2d(0.09090909090909091, 0.037037037037037035) 9
